In [ ]:
!pip install keras-tuner
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
from keras_tuner import HyperModel, RandomSearch
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint  # Ajout de ModelCheckpoint
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import LearningRateScheduler
from google.colab import drive
import os

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

def count_elements_in_subfolders(folder_path):
    if not os.path.isdir(folder_path):
        print("Le chemin spécifié n'est pas un dossier valide.")
        return

    num_elements = 0

    for root, dirs, files in os.walk(folder_path):
        for directory in dirs:
            directory_path = os.path.join(root, directory)
            elements = os.listdir(directory_path)
            num_elements_in_directory = len(elements)
            num_elements += num_elements_in_directory
            print(f"Le sous-dossier '{directory_path}' contient {num_elements_in_directory} élément(s).")

    print(f"\nAu total, les sous-dossiers du dossier '{folder_path}' contiennent {num_elements} élément(s).")

# Exemple d'utilisation
folder_path = "/content/drive/MyDrive/COURS/CC_model/dataset"
count_elements_in_subfolders(folder_path)

Le sous-dossier '/content/drive/MyDrive/COURS/CC_model/dataset/train' contient 16 élément(s).
Le sous-dossier '/content/drive/MyDrive/COURS/CC_model/dataset/test' contient 16 élément(s).
Le sous-dossier '/content/drive/MyDrive/COURS/CC_model/dataset/validation' contient 16 élément(s).
Le sous-dossier '/content/drive/MyDrive/COURS/CC_model/dataset/train/2c' contient 2240 élément(s).
Le sous-dossier '/content/drive/MyDrive/COURS/CC_model/dataset/train/1c' contient 2240 élément(s).
Le sous-dossier '/content/drive/MyDrive/COURS/CC_model/dataset/train/2e' contient 2240 élément(s).
Le sous-dossier '/content/drive/MyDrive/COURS/CC_model/dataset/train/1e' contient 2240 élément(s).
Le sous-dossier '/content/drive/MyDrive/COURS/CC_model/dataset/train/5c' contient 2240 élément(s).
Le sous-dossier '/content/drive/MyDrive/COURS/CC_model/dataset/train/5e' contient 2240 élément(s).
Le sous-dossier '/content/drive/MyDrive/COURS/CC_model/dataset/train/10c' contient 2240 élément(s).
Le sous-dossier '/co

In [ ]:
IMG_HEIGHT, IMG_WIDTH = 224, 224  # Dimensions de l'image après redimensionnement
BATCH_SIZE = 32  # Taille du lot pour le training
EPOCHS = 5  # Nombre d'époques pour le training
nb_de_classes = 16
trials = EPOCHS

In [ ]:
TRAINING_DIR = '/content/drive/MyDrive/COURS/CC_model/dataset/train'
VALIDATION_DIR = '/content/drive/MyDrive/COURS/CC_model/dataset/validation'
TEST_DIR = '/content/drive/MyDrive/COURS/CC_model/dataset/test'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 35840 images belonging to 16 classes.
Found 7680 images belonging to 16 classes.
Found 7680 images belonging to 16 classes.


In [ ]:
# Initialisation du callback EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=3)

# Ajout du callback dans la liste des callbacks pour l'entraînement
callbacks = [early_stop]

# Définition du callback ModelCheckpoint
model_checkpoint_callback = ModelCheckpoint(
    filepath='/content/drive/MyDrive/COURS/CC_model/Ely.h5',
    save_weights_only=False,  # Change to False to save the entire model
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

callbacks.append(model_checkpoint_callback)

# Ajout de la régularisation à votre modèle
class MyHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        # Définition du modèle
        base_model = MobileNet(weights='imagenet', include_top=False, input_shape=self.input_shape)
        for layer in base_model.layers[:-4]:  # Unfreeze the last 4 layers for fine-tuning
            layer.trainable = False

        model = Sequential()
        model.add(base_model)
        model.add(Flatten())
        model.add(Dense(hp.Int('units', min_value=32, max_value=512, step=32), activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))  # Tune the number of neurons
        model.add(Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))  # Tune the dropout rate
        model.add(Dense(self.num_classes, activation='softmax'))

        model.compile(
            optimizer=Adamax(
                hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG')
            ),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        return model

# Define the learning rate scheduler function
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_callback = LearningRateScheduler(scheduler)

callbacks = [early_stop, lr_callback]

In [ ]:
hypermodel = MyHyperModel(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=nb_de_classes)

tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=trials,
    directory='/content/drive/MyDrive/COURS/CC_model/Ely_hyperparm',
    project_name='trials'
)

tuner.search(train_generator, validation_data=validation_generator, epochs=EPOCHS, callbacks=callbacks)  # Ajoutez vos callbacks ici

Trial 3 Complete [00h 00m 01s]

Best val_accuracy So Far: 0.7997395992279053
Total elapsed time: 00h 22m 27s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
96                |288               |units
0                 |0.1               |dropout
1.1726e-05        |2.2032e-05        |learning_rate



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/base_tuner.py", line 270, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/base_tuner.py", line 235, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/tuner.py", line 214, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/hypermodel.py", line 144, in fit
    return model.fit(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    rais

RuntimeError: ignored

In [ ]:
# Démonter Google Drive
drive.flush_and_unmount()

# Supprimer les fichiers existants à l'emplacement de montage
!rm -rf /content/drive

# Monter Google Drive
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=trials,
    directory='/content/drive/MyDrive/COURS/CC_model/Ely_hyperparm',  # changez 'my_dir' en 'new_directory'
    project_name='trials'
)

tuner.reload()  # recharge les essais précédents
tuner.search(train_generator, validation_data=validation_generator, epochs=EPOCHS, callbacks=callbacks)  # Ajoutez vos callbacks ici

Trial 5 Complete [00h 54m 26s]
val_accuracy: 0.9595052003860474

Best val_accuracy So Far: 0.9808593988418579
Total elapsed time: 03h 37m 27s


In [ ]:
# Obtenir le meilleur modèle
best_model = tuner.get_best_models(num_models=1)[0]

history = best_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

Epoch 1/5
1120/1120 [==============================] - 651s 579ms/step - loss: 0.2561 - accuracy: 0.9692 - val_loss: 0.2274 - val_accuracy: 0.9746
Epoch 2/5
1120/1120 [==============================] - 645s 576ms/step - loss: 0.2405 - accuracy: 0.9710 - val_loss: 0.2173 - val_accuracy: 0.9797
Epoch 3/5
1120/1120 [==============================] - 652s 582ms/step - loss: 0.2273 - accuracy: 0.9737 - val_loss: 0.1950 - val_accuracy: 0.9811
Epoch 4/5
1120/1120 [==============================] - 648s 578ms/step - loss: 0.2073 - accuracy: 0.9771 - val_loss: 0.1671 - val_accuracy: 0.9885
Epoch 5/5
1120/1120 [==============================] - 658s 588ms/step - loss: 0.1931 - accuracy: 0.9778 - val_loss: 0.1682 - val_accuracy: 0.9866


In [ ]:
test_loss, test_acc = best_model.evaluate(test_generator, steps=test_generator.samples)
print('Test accuracy:', test_acc)

 240/7680 [..............................] - ETA: 24:14 - loss: 0.1778 - accuracy: 0.9820

7680/7680 [==============================] - 47s 6ms/step - loss: 0.1778 - accuracy: 0.9820
Test accuracy: 0.9820312261581421


In [ ]:
best_model.save('/content/drive/MyDrive/COURS/CC_model/mon_model_mobileNet.h5')

In [ ]:
# # Charger l'image
# img = image.load_img("/content/drive/MyDrive/COURS/CC_model/50c.png", target_size=(224, 224))

# # Convertir l'image en tableau numpy
# img_array = image.img_to_array(img)

# # Agrandir la dimension du tableau pour correspondre à l'entrée du modèle (model.input_shape)
# img_batch = np.expand_dims(img_array, axis=0)

# # Prétraiter l'image comme le modèle s'y attend
# img_preprocessed = img_batch / 255.  # Assurez-vous que cela correspond à votre prétraitement d'entraînement

# # Faire une prédiction
# prediction = best_model.predict(img_preprocessed)

# # Obtenir l'indice de la classe prédite
# predicted_class_index = np.argmax(prediction[0])

# # Obtenir le nom de la classe prédite à partir de l'indice
# predicted_class_name = list(train_generator.class_indices.keys())[list(train_generator.class_indices.values()).index(predicted_class_index)]
# print(predicted_class_name)

1/1 [==============================] - 0s 51ms/step
dict_keys(['100e', '10c', '10e', '1c', '1e', '200e', '20c', '20e', '2c', '2e', '500e', '50c', '50e', '5c', '5e', 'cheque'])
1c
